# Imports

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from numpy.linalg import slogdet, inv
import numpy as np
from dataclasses import dataclass, field
from typing import Optional, Any, Dict

# Part A

## A1. Dataset and Setup

In [ ]:
digits = load_digits()
X = digits.data    # shape (1797, 64)
y = digits.target  # labels 0–9

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## A2. Gaussian Generative Model

In [ ]:
def estimate_parameters(X, y, lam):
    N, D = X.shape #Number of samples in training data, their 64 pixel features
    K = len(np.unique(y)) # Our labels (numbers from 0 to 9) = 10

    # Priors π_k, the frequency of each label
    pi = np.zeros(K)
    for k in range(K):
        pi[k] = np.sum(y == k) / N

    # Means μ_k, calculates average of each feature that belong to that class (64 features so 64 averages)
    mu = np.zeros((K, D))
    for k in range(K):
        mu[k] = X[y == k].mean(axis=0)

    # Shared covariance Σ
    Sigma = np.zeros((D, D))
    for i in range(N):
        k = y[i]
        # this line subtracts from all the 64 training features the means we calculated of these features
        # subtract 2 rows then reshape them to column vectors
        diff = (X[i] - mu[k]).reshape(D, 1)
        # matrix multiplication that produces 64x64 matrix
        Sigma += diff @ diff.T

    Sigma /= N
    Sigma += lam * np.eye(D) # add Lambda λ only to the diagonal matrix

    return pi, mu, Sigma

In [ ]:
def compute_scores(X, pi, mu, Sigma_reg):
    N = X.shape[0]
    K = mu.shape[0]
    D = mu.shape[1]

    # Precompute inverse + log(det) for the multivariate Gaussian formula
    # logN(x;μ,Σ)= −(D/2)​log(2π) −(1/2)​log∣Σ∣ -(1/2)*(x−μ)^T * Σ^−1 * (x−μ)
    Sigma_inv = inv(Sigma_reg)
    # calculate the log of the determinant of the sigma (​log∣Σ∣)
    sign, logdet = slogdet(Sigma_reg)
    const = -0.5 * (D * np.log(2*np.pi) + logdet) #( −(D/2)​log(2π) −(1/2)​log∣Σ∣ )

    scores = np.zeros((N, K)) # liklehood of all the samples belonging to which of the 10 classes

    for k in range(K):
        diff = X - mu[k]                      # shape (N, D)
        tmp  = diff @ Sigma_inv               # shape (N, D)
        maha = np.sum(tmp * diff, axis=1)     # quadratic form
        log_gauss = const - 0.5 * maha
        scores[:, k] = np.log(pi[k]) + log_gauss

    return scores

def predict(X, pi, mu, Sigma_reg):
    scores = compute_scores(X, pi, mu, Sigma_reg)
    return np.argmax(scores, axis=1)

# Part B

# Part C

## Decision Trees

## Loading & Splitting Dataset

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)
print("Test size:", X_test.shape)

Train size: (398, 30)
Validation size: (85, 30)
Test size: (86, 30)


## Entropy calculation

In [ ]:
def entropy(y):
    vals, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs + 1e-12))

## Information Gain Calculation

In [ ]:
def information_gain(y_parent, y_left, y_right):
    n = len(y_parent)
    if len(y_left) == 0 or len(y_right) == 0:
        return 0
    return entropy(y_parent) - (len(y_left)/n)*entropy(y_left) - (len(y_right)/n)*entropy(y_right)

## Finding The Best Threshold to Split

### Finding the best threshold for a given feature

In [ ]:
def best_split_for_feature(X_column, y):
    sorted_idx = np.argsort(X_column)
    X_sorted = X_column[sorted_idx]
    y_sorted = y[sorted_idx]

    distinct = np.where(np.diff(X_sorted) != 0)[0]
    if len(distinct) == 0:
        return None, 0

    thresholds = (X_sorted[distinct] + X_sorted[distinct + 1]) / 2

    best_gain = -1
    best_threshold = None

    for thr in thresholds:
        left_mask = X_column <= thr
        y_left = y[left_mask]
        y_right = y[~left_mask]
        gain = information_gain(y, y_left, y_right)
        if gain > best_gain:
            best_gain = gain
            best_threshold = thr

    return best_threshold, best_gain

### Finding the best feature to split on

In [ ]:
def best_split_overall(X, y):
    n_features = X.shape[1]
    best = {"feature": None, "threshold": None, "gain": -1}

    for j in range(n_features):
        thr, gain = best_split_for_feature(X[:, j], y)
        if gain > best["gain"]:
            best["gain"] = gain
            best["feature"] = j
            best["threshold"] = thr

    return best

## Decision Tree Implementation

### Node Class Implementation

In [ ]:
@dataclass
class Node:
    feature: Optional[int] = None
    threshold: Optional[float] = None
    left: Any = None
    right: Any = None
    is_leaf: bool = False
    prediction: Optional[int] = None
    n_samples: int = 0
    class_counts: Dict[int, int] = field(default_factory=dict)

### DecisionTree Class Implementation

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        self.feature_importances_ = None

    def fit(self, X, y):
        n_features = X.shape[1]
        self.feature_importances_ = np.zeros(n_features)
        self.root = self._build_tree(X, y, depth=0)
        total = self.feature_importances_.sum()
        if total > 0:
            self.feature_importances_ /= total
        return self

    def _build_tree(self, X, y, depth):
        node = Node()
        node.n_samples = len(y)

        vals, counts = np.unique(y, return_counts=True)
        node.class_counts = {int(v): int(c) for v, c in zip(vals, counts)}
        node.prediction = vals[np.argmax(counts)]

        if (depth >= self.max_depth or len(y) < self.min_samples_split or len(vals) == 1):
            node.is_leaf = True
            return node

        best = best_split_overall(X, y)
        if (best["gain"] <= 0 or best["feature"] is None):
            node.is_leaf = True
            return node

        feature = best["feature"]
        threshold = best["threshold"]
        node.feature = feature
        node.threshold = threshold

        self.feature_importances_[feature] += best["gain"]

        left_mask = X[:, feature] <= threshold
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[~left_mask], y[~left_mask]

        if len(y_left) == 0 or len(y_right) == 0:
            node.is_leaf = True
            return node

        node.left = self._build_tree(X_left, y_left, depth + 1)
        node.right = self._build_tree(X_right, y_right, depth + 1)

        return node

    def _predict_one(self, x, node):
        while not node.is_leaf:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.prediction

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

# Part D